In [4]:
from GraphModels.model import CholPCorrModel

from sklearn.covariance import GraphicalLasso
from sklearn.model_selection import train_test_split

import numpy as np
import networkx as nx
from tqdm import tqdm
import warnings

from scipy.optimize import curve_fit

from collections import defaultdict

import optuna

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
def f(x, a, b, c):
    return a * np.power(x, b) + c

params = curve_fit(f, np.arange(0.1, 1, 0.1), [0.94, 0.87, 0.85, 0.77, 0.74, 0.68, 0.64, 0.55, 0.4])[0]

In [7]:
def describe(arr):
    print('Mean:', np.mean(arr))
    print('Std:', np.std(arr))
    print('Median:', np.median(arr))

In [42]:
dim = 20
n_samples = 40
density = f(0.1, *params)
num_models = 20

def optimize(trial):
    alpha = trial.suggest_float('alpha', 1e-2, 1e3, step=0.01)

    val_scores = []
    for x in range(num_models):
        model = CholPCorrModel(dim, density)
        data = model.sample(n_samples)
        data_train, data_val = train_test_split(data, test_size=0.2)

        slv = GraphicalLasso(alpha, max_iter=1000, assume_centered=True)
        slv.fit(data_train)

        val_scores.append(slv.score(data_val))

    mean, std = np.mean(val_scores), np.std(val_scores)

    trial.set_user_attr("mean", mean)
    trial.set_user_attr("std", std)

    return mean

study = optuna.create_study(direction='maximize')
study.optimize(optimize, n_trials=10, n_jobs=-1)


[I 2023-08-16 23:57:16,568] A new study created in memory with name: no-name-7d7a7585-699c-434e-9669-2e10abdb70aa


[I 2023-08-16 23:57:19,218] Trial 7 finished with value: -32.64404401650417 and parameters: {'alpha': 666.66}. Best is trial 7 with value: -32.64404401650417.
[I 2023-08-16 23:57:19,449] Trial 6 finished with value: -33.9761248134111 and parameters: {'alpha': 291.61}. Best is trial 7 with value: -32.64404401650417.
[I 2023-08-16 23:57:19,522] Trial 2 finished with value: -32.817902441596814 and parameters: {'alpha': 483.91}. Best is trial 7 with value: -32.64404401650417.
[I 2023-08-16 23:57:19,559] Trial 4 finished with value: -32.618709714599134 and parameters: {'alpha': 159.71}. Best is trial 4 with value: -32.618709714599134.
[I 2023-08-16 23:57:19,574] Trial 5 finished with value: -32.936852112567834 and parameters: {'alpha': 754.18}. Best is trial 4 with value: -32.618709714599134.
[I 2023-08-16 23:57:19,588] Trial 0 finished with value: -32.70462359423813 and parameters: {'alpha': 880.24}. Best is trial 4 with value: -32.618709714599134.
[I 2023-08-16 23:57:19,604] Trial 3 finis

In [44]:
study.best_params, study.best_trial.user_attrs

({'alpha': 217.76}, {'mean': -31.243752332046007, 'std': 1.6337629753083849})